In [2]:
from src.reader import Reader
from src.downloader import Downloader

In [ ]:
#Download images Starting from 1st challange to 10th challange
#In other words dowload 10 1st subsets
downloader = Downloader(start=0, end=10)
downloader.download(path="data/AVA.txt")